# Natural Language Processing with RNNs and Attention

In [2]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('axes',labelsize=14)
mpl.rc('xtick',labelsize=12)
mpl.rc('ytick',labelsize=12)

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR,"images",CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id,tight_layout=True,fig_extension="png",resolution=300):
    path = os.path.join(IMAGES_PATH,fig_id + "." + fig_extension)
    print("Saving Figure",fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path,format=fig_extension,dpi=resolution)

# Char-RNN

# Splitting a sequence into batches of shuffled windows 

In [3]:
np.random.seed(42)

In [4]:
tf.random.set_seed(42)

In [8]:
n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps,shift=2,drop_remainder=True)
dataset = dataset.flat_map(lambda window:window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1],window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch,Y_batch) in enumerate(dataset):
    print("_"*20,"Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("="*5, "\nY_batch")
    print(Y_batch.numpy())

W0309 00:02:08.521692 15852 deprecation.py:323] From c:\users\microsoft\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\data\util\random_seed.py:58: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


# Loading and Preparing Dataset 

In [9]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespearse.txt",shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1122304/1115394 [==============================] - 76s 68us/step


In [10]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [11]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [12]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [14]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [15]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [33]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count

In [34]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [35]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [36]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))
np.random.seed(42)
tf.random.set_seed(42)

In [37]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [38]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [39]:
dataset = dataset.prefetch(1)

In [40]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


# Creating and Training Model

In [41]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     # no dropout in stateful RNN (https://github.com/ageron/handson-ml2/issues/32)
                     # dropout=0.2, recurrent_dropout=0.2,
                     ),
    keras.layers.GRU(128, return_sequences=True,
                     # dropout=0.2, recurrent_dropout=0.2
                    ),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

Train for 31370 steps
Epoch 1/10
15288/31370 [=============>................] - ETA: 59:30 - loss: 0.9912

KeyboardInterrupt: 

# Using Model to Generate Text 

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

In [ ]:
tf.random.set_seed(42)
tf.random.categorical([[np.log(0.5),np.log(0.4),np.log(0.1)]], num_samples=40).numpy()

In [ ]:
def next_char(text,temprature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0,-1:,:]
    rescaled_logits = tf.math.log(y_proba)/temprature
    char_id = tf.random.categorical(resclaed_logits,num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
tf.random.set_seed(42)
nex_char("How are yo",temprature=1)

In [ ]:
def complete_text(text,n_chars=50,temprature):
    for _ in range(n_chars):
        text +=next_char(text,temprature)
    return text

In [ ]:
tf.random.set_seed(42)
print(complete_text("t",temprature=0.2))

In [ ]:
print(complete_text("t",temprature=1))

In [ ]:
print(complete_text("t",temprature=2))

# Stateful RNN

In [ ]:
tf.random.set_seed(42)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train])
dataset = dataset.window(window_length,shift=n_steps,drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))
dataset = dataset.map(
    lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth= max_id),Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
batch_size =32
encoded_parts = np.array_split(encoded[:train_size],batch_size)
datasets = []
for enocoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length,shift=n_steps,drop_remainder=True)
    dataset = dataset.flat_map(lambda window : window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda * windows : tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:,:-1],windows[:,1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch : (tf.one_hot(X_batch,depth = max_id),Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128,return_sequences=True,stateful=True,
                    dropout=0.2,reccurent_dropout=0.2,
                    batch_input_shape=[batch_size,None,max_id]),
    keras.layers.GRU(128,return_sequences=True,stateful=True,
                     dropout=0.2,recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
                     
])

In [ ]:
class ResetStatesCallBack(keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam")
steps_per_epoch = train_size// batch_size //n_steps
model.fit(dataset,steps_per_epoch=steps_per_epoch,epochs=50,
         callbacks=[ResetStateCallBack()])